In [1]:
import pandas as pd
import traceback

In [2]:
#EXTRACT

dfs = []
for i in range (1,9):
    for j in range(1,3):
            for l in range(1,3):
                try:
                    df = pd.read_excel(f"../data/isruc-sleep/subgroupII/{i}/{j}/{j}_{l}.xlsx", usecols="A:I")
                    if not "spo2" in [str(col).lower() for col in df.columns]:
                        print(f"../data/isruc-sleep/subgroupII/{i}/{j}/{j}_{l}.xlsx has no header.")
                        df = pd.read_excel(f"../data/isruc-sleep/subgroupII/{i}/{j}/{j}_{l}.xlsx", usecols="A:I", header=None)
                        
                    
                    df["patient_id"] = i
                    df["patient_vist_id"]= j
                    df['score_id']= l
                    dfs.append(df)
                except Exception as e:
                    print(f"Error parsing ../data/isruc-sleep/subgroupII/{i}/{j}/{k}_{l}.xlsx: {e}")

../data/isruc-sleep/subgroupII/1/2/2_2.xlsx has no header.
../data/isruc-sleep/subgroupII/2/2/2_2.xlsx has no header.
../data/isruc-sleep/subgroupII/3/2/2_2.xlsx has no header.
../data/isruc-sleep/subgroupII/4/2/2_2.xlsx has no header.
../data/isruc-sleep/subgroupII/5/1/1_2.xlsx has no header.
../data/isruc-sleep/subgroupII/5/2/2_2.xlsx has no header.
../data/isruc-sleep/subgroupII/6/1/1_2.xlsx has no header.
../data/isruc-sleep/subgroupII/6/2/2_2.xlsx has no header.
../data/isruc-sleep/subgroupII/7/1/1_2.xlsx has no header.
../data/isruc-sleep/subgroupII/7/2/2_2.xlsx has no header.
../data/isruc-sleep/subgroupII/8/1/1_2.xlsx has no header.
../data/isruc-sleep/subgroupII/8/2/2_2.xlsx has no header.


In [3]:
# TRANSFORM
def transform(df):
    df = df.copy()
    df.columns=["epoch","stage","spo2","hr","events","bpos","txln","txex","technote", "patient_id","patient_visit_id","score_id"]

    #no point keeping points if scorer didnt score
    df.dropna(how="any", subset=['epoch','stage'], inplace=True)
    df.dropna(how="all", inplace=True)
    df.loc[df.technote.isna(), "technote"]=""
    df.loc[df.events.isna(),'events']= ""
    df['events'] = df.apply(lambda r: r['events'] + " ".join(r['hr'].split(" ")[1:]) if isinstance(r['hr'],str) else r['events'], axis=1)
    df['hr'] = df.hr.apply(lambda r: float(r.split(" ")[0]) if isinstance(r, str) else float(r))
    df.epoch = df.epoch.astype("int32")
    return df

In [4]:
df = pd.concat([transform(df) for df in dfs])

In [5]:
df.head()


,epoch,stage,spo2,hr,events,bpos,txln,txex,technote,patient_id,patient_visit_id,score_id
0,1,W,0.0,95.0,L Out,B,0.0,5.0,,1,1,1
1,2,W,97.0,89.0,,B,0.0,5.0,,1,1,1
2,3,W,0.0,95.0,,B,0.0,5.0,,1,1,1
3,4,W,0.0,87.0,,B,0.0,5.0,,1,1,1
4,5,W,0.0,87.0,MChg,B,0.0,5.0,,1,1,1


In [6]:
df.to_csv("../data/isruc-sleep/subgroupII/scoring.csv")

In [7]:
studyII = pd.read_excel(f"../data/isruc-sleep/subgroupII/Details_subgroup_II_Submission.xlsx", header=2,usecols="A:R")

In [8]:
studyII.columns=[x.lower() for x in studyII.columns]


In [9]:
studyII['age']=pd.to_numeric(studyII.age,errors="coerce")
studyII['date of recording'] = pd.to_datetime(studyII['date of recording'])

In [10]:
studyII.head()

,subject,age,sex,diagnosis,other problems,epoches,w%,n1%,n2%,n3%,rem%,w%.1,n1%.1,n2%.1,n3%.1,rem%.1,cohen’s kappa,date of recording
0,1_Rec.1,70.0,M,SAOS,Central apneas; split-night study,933.0,14.36,18.33,21.97,25.08,20.26,15.524625,11.027837,29.871520,25.053533,18.522484,0.875618,2009-01-06
1,1_Rec.2,NaN,NaN,NaN,NaN,787.0,47.65,15.88,10.42,16.14,9.91,44.599746,24.015248,10.292249,12.960610,8.132147,0.898454,2012-03-02
2,2_Rec.1,79.0,M,SAOS,split-night study,851.0,44.18,18.68,24.91,6.46,5.76,54.342723,10.915493,22.183099,7.981221,4.577465,0.674286,2011-04-28
3,2_Rec.2,NaN,NaN,NaN,NaN,869.0,34.98,7.71,24.86,21.29,11.16,34.637514,17.261220,21.979287,16.800921,9.321059,0.959238,2012-09-14
4,3_Rec.1,43.0,F,SAOS,NaN,871.0,8.96,18.71,38.00,20.09,14.24,10.779817,19.266055,33.142202,19.266055,17.545872,0.884377,2011-05-16


In [11]:
studyII.to_csv("../data/isruc-sleep/subgroupII/scoring_summary.csv")